# QARP Model

### 1. Compute Factors

In [1]:
# import libraries
import pandas
import numpy as np
import datetime

In [75]:
# helper functions

""" Gets the lagged difference for each company. """
def diffCol(col, gvkey, step):
    return stepLookback(col, gvkey, step, lambda x: x["col"] - x["shiftedCol"])

""" Gets a binary column of whether the lagged and current values are negative. """
def negLookback(col, gvkey, step):
    return stepLookback(col, gvkey, step, lambda x: 1 if x["col"] < 0 and x["shiftedCol"] < 0 else 0)

""" Gets a column of the sum of absolute values of lagged and current values. """
def absAddition(col, gvkey, step):
    return stepLookback(col, gvkey, step, lambda x: abs(x["col"]) + abs(x["shiftedCol"]))

""" Applies func() to col[i] and col[i-step], if they have the same gvkey. """
def stepLookback(col, gvkey, step, func):
    dt = col.shift(step).to_frame("shiftedCol")
    dt["gvkeyDiff"] = gvkey.diff(periods=step)
    dt["col"] = col
    dt["diffCol"] = dt.apply(lambda x: func(x) if x["gvkeyDiff"] == 0 else float("NaN"), axis=1)
    
    return dt["diffCol"]

In [3]:
# Import csv data
compustat = pandas.read_csv("Compustat19612018.csv")
# Used dtypes here to speed up function
crsp = pandas.read_csv("CRSP1960_2018.csv", \
    dtype={'PERMNO': int, 'date': str, 'EXCHCD': float, 'SICCD': str, 'SHRCLS': str, 'PRIMEXCH': str, \
           'PRC': float, 'RET': str, 'SHROUT': float, 'CFACSHR': float, 'vwretd': float, 'sprtrn': float})
cpi = pandas.read_csv("CPIData.csv")
linking_table = pandas.read_csv('CompustatMergedDatabase.csv')

#### Link CRSP and Compustat

In [4]:
# Fix LINKDT and LINKENDDT dates
linking_table.LINKDT = pandas.to_datetime(linking_table.LINKDT.astype(str), format='%Y%m%d', errors='coerce').fillna(datetime.date.today())
linking_table.LINKENDDT = pandas.to_datetime(linking_table.LINKENDDT.astype(str), format='%Y%m%d', errors='coerce').fillna(datetime.date.today())

In [18]:
# Merge compustat and linking_table based on gvkey to get permnos for each compustat company
compustat = compustat.merge(linking_table, on='gvkey')

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'curcd', 'act', 'at', 'capx', 'ceq', 'che', 'cogs', 'csho',
       'dlc', 'dltt', 'dp', 'ebit', 'ib', 'lct', 'lt', 'mib', 'mibt', 'pi',
       'pstk', 'pstkl', 'pstkrv', 're', 'revt', 'sale', 'seq', 'tie', 'txp',
       'txt', 'costat', 'conm_x', 'LINKPRIM_x', 'LIID_x', 'LINKTYPE_x',
       'LPERMNO_x', 'LPERMCO_x', 'LINKDT_x', 'LINKENDDT_x', 'permnoyear',
       'conm_y', 'LINKPRIM_y', 'LIID_y', 'LINKTYPE_y', 'LPERMNO_y',
       'LPERMCO_y', 'LINKDT_y', 'LINKENDDT_y', 'conm', 'LINKPRIM', 'LIID',
       'LINKTYPE', 'LPERMNO', 'LPERMCO', 'LINKDT', 'LINKENDDT'],
      dtype='object')


In [6]:
# Create year column in crsp to link with permno
crsp['year'] = pandas.to_datetime(crsp.date, format='%m/%d/%Y').dt.year;

In [7]:
# Create permnoyear column for crsp and compustat
crsp['permnoyear'] = crsp.PERMNO.map(str) + crsp.year.map(str)
compustat['permnoyear'] = compustat.LPERMNO.map(str) + compustat.fyear.map(str)

In [8]:
# For each year, we need the most recent value, so I am goruping it by permnoyear and then
# just taking the tail (last member) of the group. Its quick and dirty and we probably want 
# a better way to do this so we know we are getting the last value if the data isn't sorted
crspGetLastYear = crsp[['permnoyear','EXCHCD','SICCD','SHROUT' ,'SHRCLS','PRIMEXCH', 'date']]
crspGetLastYear = crspGetLastYear.groupby('permnoyear').tail(1)

In [9]:
# These columns need to be summed for each year, which is being done here
# RET is a str column and sometimes has error characters so still need to handle that
crspSum = crsp[['permnoyear', 'PRC', 'RET', 'vwretd', 'sprtrn']]
crspSum = crspSum.groupby(by=['permnoyear'])['sprtrn', 'PRC', 'RET', 'vwretd', 'sprtrn'].sum()

In [19]:
# Merge the two annualized subset dataframes back together
crsp = crspSum.merge(crspGetLastYear, on='permnoyear')

Index(['permnoyear', 'sprtrn', 'PRC', 'vwretd', 'sprtrn', 'EXCHCD', 'SICCD',
       'SHROUT', 'SHRCLS', 'PRIMEXCH', 'date'],
      dtype='object')


In [20]:
# Merge crsp and compustat by permnoyear and convert permnoyear to ints
model_data = compustat.merge(crsp, on='permnoyear')
model_data["permnoyear"] = pandas.to_numeric(model_data["permnoyear"])
print(model_data.columns)

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'curcd', 'act', 'at', 'capx', 'ceq', 'che', 'cogs', 'csho',
       'dlc', 'dltt', 'dp', 'ebit', 'ib', 'lct', 'lt', 'mib', 'mibt', 'pi',
       'pstk', 'pstkl', 'pstkrv', 're', 'revt', 'sale', 'seq', 'tie', 'txp',
       'txt', 'costat', 'conm_x', 'LINKPRIM_x', 'LIID_x', 'LINKTYPE_x',
       'LPERMNO_x', 'LPERMCO_x', 'LINKDT_x', 'LINKENDDT_x', 'permnoyear',
       'conm_y', 'LINKPRIM_y', 'LIID_y', 'LINKTYPE_y', 'LPERMNO_y',
       'LPERMCO_y', 'LINKDT_y', 'LINKENDDT_y', 'conm', 'LINKPRIM', 'LIID',
       'LINKTYPE', 'LPERMNO', 'LPERMCO', 'LINKDT', 'LINKENDDT', 'sprtrn',
       'PRC', 'vwretd', 'sprtrn', 'EXCHCD', 'SICCD', 'SHROUT', 'SHRCLS',
       'PRIMEXCH', 'date'],
      dtype='object')


#### Profitability Factors
1. gross profits over assets (GPOA) = (Revenue - costs of goods sold) / total assets
2. return on equity (ROE) = net income / book-equity 
3. return on assets (ROA) = net income / total assets
4. cash flow over assets (CFOA) = (net income + depreciation - changes in working capital - capital expenditures) / total assets
5. gross margin (GMAR) = (revenue - cost of goods sold) / total sales
6. low accruals (ACC) = - (change in working capital - depreciation) / total assets


In [12]:
# GPOA
gp = model_data.revt - model_data.cogs
gpoa = gp / model_data["at"]

# ROE
be = (model_data.seq - model_data.pstk).fillna(model_data.ceq + model_data.pstk).fillna(model_data["at"] - model_data["lt"] + compustat["mibt"])
roe = model_data["ib"] / be

# ROA
roa = model_data["ib"] / model_data["at"]

# CFOA
wc = model_data["act"] - model_data["lct"] - model_data["che"] + model_data["dlc"] + model_data["txp"]
wcDiff = diffCol(wc, model_data["gvkey"], 1)
cf = model_data["ib"] + model_data["dp"] - wcDiff - model_data["capx"]
cfoa = cf / model_data["at"]

# GMAR
gmar = gp / model_data["sale"]

# ACC
acc = -wcDiff / model_data["at"]

#### Growth Factors
Five year growth of profitability factors

In [13]:
# Store for calculating later
ib = model_data["ib"]
diffIb = diffCol(ib, model_data["gvkey"], 5)
diffGp = diffCol(gp, model_data["gvkey"], 5)
atShift = model_data["at"].shift(5)
ceqShift = model_data["ceq"].shift(5)

# Growth factors
delGpoa = diffGp / atShift
delRoe = diffIb / ceqShift
delRoa = diffIb / atShift
delCfoa = diffCol(cf, model_data["gvkey"], 5) / ceqShift
delGpoa = diffGp / ceqShift

#### Safety Factors

1. low beta (BAB) = cov(value-weighted return, S&P 500 return) / var(S&P 500 return)
2. low leverage (LEV) = - (total debt) / total assets
3. Ohlson’s O-score (OSCORE) = - (-1.32 - 0.407 * log(ADJASSET/CPI) + 6.03 * TLTA - 1.43 * WCTA + 0.076 * CLCA - 1.72 * OENEG - 2.37 * NITA - 1.83 * FUTL + 0.285 * INTWO - 0.521 * CHIN)
    *  Adjusted Total Assets (ADJASSET) = total assets + 10% * (Market equity - Book Equity)
    * Consumer Price Index (CPI)
    * Book Value of Debt (TLTA) = book value of debt / ADJASSET
    * Working Capital to Assets (WCTA) = (current assets - current liabilities) / ADJASSET
    * Current Liabilities to Assets (CLCA) = current liabilities / current assets
    * OENEG = 1 if total liabilities exceed total assets
    * Net income to asssets (NITA) = net income / total assets
    * Pretax Income to Liabilities(FUTL) = pretax income / total liabilites
    * INTWO = 1 if net income is negative for the current and prior fiscal year
    * (CHIN) = change in net income
4. Altmans Z-Score (AZSCORE) = (1.2 Working Capital + 1.4 Retained Earnings + 3.3 EBIT + 0.6 Market Cap + Sales) / Total Assets
5. low ROE volatility (EVOL) = Standard deviation of quarterly ROE over the past 60 quarters or 5 years (if quarterly null)


In [80]:
# Safety factors
bab = 0
lev = -(model_data["dltt"] + model_data["dlc"] + model_data["mibt"] + model_data["pstk"]) / model_data["at"]

# Oholn's O-score
adjasset = model_data["at"] + 0.1*(model_data["csho"] * model_data["PRC"] - model_data["ceq"])
tlta = (model_data["dlc"] + model_data["dltt"]) / adjasset
wcta = (model_data["act"] - model_data["lct"]) / adjasset
clca = model_data["lct"] / model_data["act"]
oeneg = model_data["lt"] - model_data["at"]
oeneg[oeneg <= 0] = 0
nita = model_data["ib"] / model_data["at"]
futl = model_data["pi"] / model_data["lt"]
intwo = negLookback(model_data["ib"], model_data["gvkey"], 1)
chin = diffCol(model_data["ib"], model_data["gvkey"], 1) / absAddition(model_data["ib"], model_data["gvkey"], 1)
oscore = -(-1.32 - 0.407 * np.log(adjasset / 1) + 6.03 * tlta - 1.43 * wcta + 0.076 * clca \
           - 1.72 * oeneg - 2.37 * nita - 1.83 * futl + 0.285 * intwo - 0.521 * chin)  # Dividing by 1 instead of CPI

# Altman's Z-score
ebit = model_data["ib"] + model_data["tie"] + model_data["txt"]
me = model_data["csho"] * model_data["PRC"]
azscore = (1.2 * wc + 1.4 * model_data["re"] + 3.3 * ebit + 0.6 * me) / model_data["at"]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log


35728
